# 04 — Simulate Returns, Compute VaR/CVaR, and Stress Test

**Objective**
- Map **simulated uniforms** (from the Gaussian copula) back to **returns** using inverse t-CDF.
- Build an **equal-weight portfolio** and compute **VaR/CVaR** at 95% & 99%.
- Compare **Copula-based**, **Historical**, and **Gaussian parametric** risk.
- Run a **conditional stress test** (fix XLK = −5%) and analyze impact.
- Save figures and result tables under `figures/` and `data/processed/`.


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
from scipy import stats
import joblib

from quantcopula.data import get_prices
from quantcopula.margins import fit_student_t_params, from_uniforms, frame_to_params
from quantcopula.copula import sample_copula
from quantcopula.risk import (
    var_cvar,
    portfolio_equal_weight,
    gaussian_parametric_var_cvar,
)
from quantcopula.plotting import (
    plot_hist,
    plot_ecdf_with_lines,
)

# Paths & constants
os.makedirs("data/processed", exist_ok=True)
os.makedirs("figures", exist_ok=True)

ALPHA_LIST = [0.95, 0.99]
N_SAMPLES  = 10_000  # for extra simulations if needed


In [ ]:
# Load previously saved artifacts from notebooks 01–03
prices = get_prices(
    ["XLK", "XLF", "XLI", "XLV"],
    "2013-01-01",
    "2023-12-31",
    cache_path="data/raw/prices.parquet",
    auto_adjust=False,
).dropna(how="any")
log_returns = np.log(prices / prices.shift(1)).dropna(how="any")

# Fitted Student-t parameters
params_df = pd.read_csv("data/processed/t_params.csv")
params = frame_to_params(params_df)

# Empirical uniforms (from PIT)
U_empirical = pd.read_parquet("data/processed/uniforms.parquet")

# Fitted Gaussian copula
copula = joblib.load("data/processed/gaussian_copula.pkl")

# Simulated uniforms (from 03), or regenerate if desired
U_sim = pd.read_parquet("data/processed/u_simulated.parquet")
U_sim = U_sim[U_empirical.columns]  # ensure consistent column order
U_sim.head()


## Inverse PIT
Convert uniforms back to returns using the fitted t parameters for each asset.


In [ ]:
simulated_returns = from_uniforms(U_sim, params)
simulated_returns.to_parquet("data/processed/simulated_returns.parquet")
"data/processed/simulated_returns.parquet"


## Portfolio VaR/CVaR
Compute risk on:
- **Copula-based simulations** (non-parametric VaR/CVaR)
- **Historical returns** (non-parametric VaR/CVaR)
- **Gaussian parametric** (μ, σ fitted on historical)


In [ ]:
port_sim  = portfolio_equal_weight(simulated_returns)
port_hist = portfolio_equal_weight(log_returns)

def risk_table(port_series, label):
    rows = []
    for a in ALPHA_LIST:
        v, c = var_cvar(port_series.values, a)
        rows.append({"Model": label, "Alpha": a, "VaR": v, "CVaR": c})
    return pd.DataFrame(rows)

tbl_cop = risk_table(port_sim,  "Copula-Based")
tbl_hist= risk_table(port_hist, "Historical")

# Gaussian parametric on historical
rows_g = []
for a in ALPHA_LIST:
    v, c = gaussian_parametric_var_cvar(port_hist, a)
    rows_g.append({"Model": "Gaussian (Hist)", "Alpha": a, "VaR": v, "CVaR": c})
tbl_gauss = pd.DataFrame(rows_g)

risk_comparison = pd.concat([tbl_cop, tbl_hist, tbl_gauss], ignore_index=True)
risk_comparison.to_csv("data/processed/risk_comparison_long.csv", index=False, float_format="%.6f")
risk_comparison.pivot(index="Model", columns="Alpha", values=["VaR", "CVaR"]).round(4)


In [ ]:
# Histogram (simulated portfolio)
plot_hist(port_sim, "figures/portfolio_hist.png", "Simulated Portfolio Return (10k scenarios)")

# ECDF with VaR/CVaR markers (95% & 99%)
sorted_ret = np.sort(port_sim.values)
ecdf = np.arange(1, len(sorted_ret) + 1) / len(sorted_ret)

v95, c95 = var_cvar(port_sim.values, 0.95)
v99, c99 = var_cvar(port_sim.values, 0.99)
plot_ecdf_with_lines(sorted_ret, ecdf, {
    "VaR 95%": v95, "CVaR 95%": c95, "VaR 99%": v99, "CVaR 99%": c99
}, out_path="figures/portfolio_ecdf.png")


## Conditional Stress Test: Fix **XLK = −5%**
1. Convert −5% to **U-space** via the fitted t-CDF for XLK.
2. Sample uniforms from the copula and **override** `U["XLK"]` with that fixed value.
3. Inverse PIT back to returns, build portfolio, and recompute VaR/CVaR.


In [ ]:
asset = "XLK"
# Map −5% to U-space using t CDF and fitted params
df, loc, scale = params[asset]
u_fixed = stats.t.cdf(-0.05, df, loc=loc, scale=scale)

# Generate new uniforms and fix XLK dimension
U_cond = sample_copula(copula, N_SAMPLES, U_empirical.columns)
U_cond[asset] = float(u_fixed)

# Back to returns
cond_returns = from_uniforms(U_cond, params)
port_cond = portfolio_equal_weight(cond_returns)

# Risk under stress
v95_c, c95_c = var_cvar(port_cond.values, 0.95)
v99_c, c99_c = var_cvar(port_cond.values, 0.99)

pd.DataFrame({
    "Metric": ["VaR 95%", "CVaR 95%", "VaR 99%", "CVaR 99%"],
    "Baseline": [v95, c95, v99, c99],
    "Conditional (XLK=-5%)": [v95_c, c95_c, v99_c, c99_c],
}).round(4)


In [ ]:
plot_hist(port_cond, "figures/portfolio_hist_conditional.png",
          "Portfolio Return (Conditioned on XLK = −5%)")


## Saved Artifacts
- `data/processed/simulated_returns.parquet`
- `data/processed/risk_comparison_long.csv`
- `figures/portfolio_hist.png`
- `figures/portfolio_ecdf.png`
- `figures/portfolio_hist_conditional.png`

## Takeaways
- Copula-based simulation captures dependence beyond simple Gaussian parametrics.
- VaR/CVaR (copula) typically shows heavier tails than Gaussian (depending on data).
- Conditioning **XLK = −5%** deepens left-tail risk — useful for scenario analysis.
